In [3]:
import pandas as pd
import bz2
from sqlalchemy import create_engine
database_url = 'mysql://localhost:3306/wikipedia'
engine = create_engine(database_url)
date_stamp = '202511'

In [4]:
pageviews_raw = pd.read_csv('/Users/rickmark/Downloads/pageviews-202511-user.csv', sep=' ',  names=['project', 'page_title', 'page_id', 'access_method', 'view_count', 'view_sequence'])

In [6]:
df = pageviews_raw.drop(['project', 'access_method', 'view_sequence', 'page_title'], axis=1)

df = df[df['page_id'].notnull()]

df = df.groupby('page_id').agg({'view_count': 'sum'}).reset_index()
df['ds'] = f"{date_stamp[0:4]}-{date_stamp[4:6]}-01"
df['page_id'] = df['page_id'].astype(int)
df

,page_id,view_count,ds
0,1,9786902,2025-11-01
1,2,3137836,2025-11-01
2,3,1160586,2025-11-01
3,4,1718610,2025-11-01
4,5,68004,2025-11-01
...,...,...,...
79788283,179229279,1,2025-11-01
79788284,179229289,1,2025-11-01
79788285,179229290,2,2025-11-01
79788286,179229296,1,2025-11-01


In [7]:
df.to_sql(name='pageviews_raw', con=engine, if_exists='replace', index=False)

79788288

In [ ]:
%%sql
CREATE TABLE mysql.wikipedia.pagview_rollup AS
WITH pageview_rollup AS (SELECT page_id, SUM(view_count) AS view_count
                         FROM mysql.wikipedia.pageviews_raw AS pr
                         GROUP BY page_id)
SELECT p.page_id, pr.view_count
FROM pageview_rollup AS pr
INNER JOIN mysql.wikipedia.page AS p
    ON p.page_id = pr.page_id
INNER JOIN mysql.wikipedia.target_entities AS te
    ON te.page_id = p.page_id

In [10]:
%%sql
WITH pageview_rollup AS (SELECT page_id, SUM(view_count) AS view_count
                         FROM mysql.wikipedia.pageviews_raw AS pr
                         GROUP BY page_id)
SELECT p.page_id, pr.view_count
FROM pageview_rollup AS pr
INNER JOIN mysql.wikipedia.page AS p
    ON p.page_id = pr.page_id
INNER JOIN mysql.wikipedia.target_entities AS te
    ON te.page_id = p.page_id

,page_id,view_count


In [13]:
%%sql
SELECT * FROM mysql.wikipedia.pageviews_raw

,page_id,view_count,ds
0,1,9786902,2025-11-01
1,2,3137836,2025-11-01
2,3,1160586,2025-11-01
3,4,1718610,2025-11-01
4,5,68004,2025-11-01
...,...,...,...
79788283,179229279,1,2025-11-01
79788284,179229289,1,2025-11-01
79788285,179229290,2,2025-11-01
79788286,179229296,1,2025-11-01


In [14]:
%%sql
SELECT page_id, SUM(view_count) AS view_count
FROM mysql.wikipedia.pageviews_raw AS pr
GROUP BY page_id

,page_id,view_count
0,406525.0,534.0
1,186904.0,1594.0
2,213928.0,4150.0
3,82616.0,5366.0
4,187449.0,617.0
...,...,...
79788284,94882028.0,1.0
79788285,94882200.0,1.0
79788286,94882859.0,11.0
79788287,7490116.0,13.0
